#### Load Libraries

In [8]:
# Pandas and numpy
import pandas as pd
import numpy as np

# On top of the required libraries for this course,
# I installed in my 'ysc4224 environment' additional libraries:
# matplotlib and seaborn for data visualisation,
# and scipy and sklearn for automating some EDA tasks.

import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sklearn as sklearn

# openpyxl - dependency for reading excel
import openpyxl

### Load Data

In [9]:
df = pd.read_csv('Telco-Customer-Churn.csv')
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


### Exploratory Data Analysis

You can leverage what done in one of the previous assignments, recreate it and expand it.

#### Data cleaning

Taking these decisions into account, let's start data cleaning:

In [19]:
# Cleaning items with str value ' ' in charges_total and with gender as 'Prefer not to say'
data = df
data['TotalCharges'] = data['TotalCharges'].replace(' ', 0)
data = df.drop(index = df[df['gender']=='Prefer not to say'].index, axis=0)
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


#### Making categorical data consistent

In [20]:
data = data.replace({'Yes' : 1, 'No' : 0, 'No phone service' : 0, 'No internet service' : 0, 'Female' : 0, 'Male' : 1})
data['InternetService'] = df['InternetService'] # 'No' values in this column need to be intact for a one-hot encoder to work.
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,0,DSL,0,...,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,0,DSL,1,...,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,0,DSL,1,...,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,0,DSL,1,...,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,0,Fiber optic,0,...,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,1,1,24,1,1,DSL,1,...,1,1,1,1,One year,1,Mailed check,84.80,1990.5,0
7039,2234-XADUH,0,0,1,1,72,1,1,Fiber optic,0,...,1,0,1,1,One year,1,Credit card (automatic),103.20,7362.9,0
7040,4801-JZAZL,0,0,1,1,11,0,0,DSL,1,...,0,0,0,0,Month-to-month,1,Electronic check,29.60,346.45,0
7041,8361-LTMKD,1,1,1,0,4,1,1,Fiber optic,0,...,0,0,0,0,Month-to-month,1,Mailed check,74.40,306.6,1


In [21]:
data.dtypes

customerID           object
gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService      object
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract             object
PaperlessBilling      int64
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

#### Standardisation

For our later data visualisations, and for the purposes of model-building or other activities that may be sensitive to Euclidean distance, let's proceed to make versions of `data` and `data_ohe` with values standardised.

In [23]:
# Using sklearn StandardScaler
from sklearn import preprocessing
scale = sklearn.preprocessing.StandardScaler()

# Standardising data_sd
data_sd = data
data_sd['tenure'] = scale.fit_transform(data[['tenure']])
data_sd['MonthlyCharges'] = scale.fit_transform(data[['MonthlyCharges']])
data_sd['TotalCharges'] = scale.fit_transform(data[['TotalCharges']])

data_sd

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,-1.277445,0,0,DSL,0,...,0,0,0,0,Month-to-month,1,Electronic check,-1.160323,-0.992611,0
1,5575-GNVDE,1,0,0,0,0.066327,1,0,DSL,1,...,1,0,0,0,One year,0,Mailed check,-0.259629,-0.172165,0
2,3668-QPYBK,1,0,0,0,-1.236724,1,0,DSL,1,...,0,0,0,0,Month-to-month,1,Mailed check,-0.362660,-0.958066,1
3,7795-CFOCW,1,0,0,0,0.514251,0,0,DSL,1,...,1,1,0,0,One year,0,Bank transfer (automatic),-0.746535,-0.193672,0
4,9237-HQITU,0,0,0,0,-1.236724,1,0,Fiber optic,0,...,0,0,0,0,Month-to-month,1,Electronic check,0.197365,-0.938874,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,1,1,-0.340876,1,1,DSL,1,...,1,1,1,1,One year,1,Mailed check,0.665992,-0.127605,0
7039,2234-XADUH,0,0,1,1,1.613701,1,1,Fiber optic,0,...,1,0,1,1,One year,1,Credit card (automatic),1.277533,2.242606,0
7040,4801-JZAZL,0,0,1,1,-0.870241,0,0,DSL,1,...,0,0,0,0,Month-to-month,1,Electronic check,-1.168632,-0.852932,0
7041,8361-LTMKD,1,1,1,0,-1.155283,1,1,Fiber optic,0,...,0,0,0,0,Month-to-month,1,Mailed check,0.320338,-0.870513,1


# Notes for the team

`df` - original dataset, untouched<br>
`data` - cleaned dataset<br>
`data_sd` - standardised cleaned dataset<br>

### Feature Engineering and Analysis
Think about any possible feature that is predictive of the target.  
Measure how each feature is predictive of the target and select all those relevant for the model.

### Configure the model using an appropriate classification technique

### Chose the correct metric for assessment of the model